# 메모 프로그램 구현하기
<hr></hr>

우리가 구현해보고자 하는 프로그램은 윈도우 운영체제에 제공되는 "스티커 메모" 프로그램 입니다. 물론 현재 버전의 윈도우용 "스티커메모" 프로그램은 간단해 보이지만 많은 기능이 탑재된 메모 프로그램 입니다만 우리는 지금까지 배운 sqlite 와 PySide2 를 활용하여 비슷하게 흉내정도 내보는 수준까지를 목적으로 삼고 있습니다.

<img src="images/13.jpg" style="border:1px solid #000000"><br>
*<center>-Python 으로 만든 메모 프로그램 미리보기 입니다.-</center>*

- 윈도우용 스티커메모 프로그램처럼 외관이 포스트잇 같은 형태로 제작해야 합니다.
- 메모창의 배경색상을 각각 설정할 수 있게 합니다.
- 메모의 창 위치 정보, 창 크기 정보, 메모의 내용은 sqlite 를 활용하여 데이터베이스에 저장합니다.
- 메모 프로그램이 실행되면 기존의 메모 정보를 sqlite 에서 불러오기 하여 마지막 정보값을 그대로 불러올 수 있어야 합니다.

### 기본 메모 위젯 작성

In [ ]:
from PySide2 import QtCore, QtGui, QtWidgets
import sys

class MyMemo(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        self.color_text = "#000000"
        self.color_bg = "#dce459"
        self.initUI()

    def initUI(self):
        self.note = QtWidgets.QTextEdit(self)
        color_text = "QTextEdit {border: 0; background-color: " + self.color_bg + "; color: " + self.color_text + "}"
        self.note.setStyleSheet(color_text)
        self.layout = QtWidgets.QVBoxLayout()
        self.layout.setSpacing(0)
        self.layout.setMargin(0)
        self.layout.addWidget(self.note)
        self.setLayout(self.layout)
        self.show()

app = QtWidgets.QApplication(sys.argv)
memo = MyMemo()
sys.exit(app.exec_())

<img src="images/14.jpg" style="border:1px solid #000000"><br>
코드를 실행하면 위의 이미지 처럼 기본적인 메모창이 실행되는걸 볼 수 있습니다. 

- ```self.color_text = "#000000"```, ```self.color_bg = "#dec459"``` 로 메모 창의 기본 글자색과 기본 배경색을 설정합니다.
- PySide 에서는 스타일스트 형식의 코드를 지원하기 때문에```self.note.setStyleSheet(color_text)``` 을 통해 color_text 변수에 담긴 스타일을 쉽게 적용할 수 있습니다.
- ```QVBoxLayout()``` 레이아웃을 생성하여 마진과 공백을 0으로 설정한 후 QTextEdit 위젯을 추가한 후 윈도우 전체 레이아웃으로 설정합니다.

위 이미지를 보면 윈도우에 python 이라고 표기된 기본 타이틀바가 표기되는것을 볼 수 있습니다. 우리가 만들 메모프로그램은 포스트잇 같은 디자인을 생각했기 때문에 이를 제거해야 합니다. 그래서 우리는 이를 제거하고 몇가지 기능을 수행할 수 있는 버튼을 추가한 새로운 타이틀바를 생성해서 대체 시켜야 합니다.

### 메모 위젯에 커스텀 타이틀바 생성하기

<img src="images/15.jpg" style="border:1px solid #000000"><br>
타이틀바는 각 위젯을 가로로 배치하기 위해서 QHBoxLayout 을 사용하여 QPushButton, QLabel, QPushButton 을 위의 이미지처럼 구성 하도록 합니다.

In [ ]:
from PySide2 import QtCore, QtGui, QtWidgets
import sys


class MyBar(QtWidgets.QWidget):
    def __init__(self, parent):
        super(MyBar, self).__init__()
        self.parent = parent
        self.layout = QtWidgets.QHBoxLayout()
        self.layout.setMargin(0)
        self.layout.setSpacing(0)
        self.title = QtWidgets.QLabel()

        btn_size = 30

        self.btn_new = QtWidgets.QPushButton("+")
        self.btn_new.setFixedSize(btn_size, btn_size)
        self.btn_new.setStyleSheet("border:0px; background-color: {};".format(parent.color_bg))

        self.btn_close = QtWidgets.QPushButton("x")
        self.btn_close.setFixedSize(btn_size, btn_size)
        self.btn_close.setStyleSheet("border:0px; background-color: {};".format(parent.color_bg))

        self.btn_color = QtWidgets.QPushButton("c")
        self.btn_color.setFixedSize(btn_size, btn_size)
        self.btn_color.setStyleSheet("border:0px; background-color: {};".format(parent.color_bg))

        self.title.setFixedHeight(30)
        self.title.setAlignment(QtCore.Qt.AlignCenter)
        self.layout.addWidget(self.btn_new)
        self.layout.addWidget(self.title)
        self.layout.addWidget(self.btn_color)
        self.layout.addWidget(self.btn_close)

        self.title.setStyleSheet("""
            background-color: {};
            color: {};
        """.format(parent.color_bg, parent.color_text))

        self.setLayout(self.layout)


class MyMemo(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        self.color_text = "#000000"
        self.color_bg = "#dce459"
        self.initUI()

    def initUI(self):
        self.note = QtWidgets.QTextEdit(self)
        color_text = "QTextEdit {border: 0; background-color: " + self.color_bg + "; color: " + self.color_text + "}"
        self.note.setStyleSheet(color_text)

        # 타이틀바 생성
        self.title_bar = MyBar(self)
        self.title_bar.setStyleSheet("top: -20px")

        self.layout = QtWidgets.QVBoxLayout()
        self.layout.setSpacing(0)
        self.layout.setMargin(0)

        self.layout.addWidget(self.title_bar)
        self.layout.addWidget(self.note)
        self.setLayout(self.layout)
        self.setWindowFlags(QtCore.Qt.FramelessWindowHint)
        self.show()


app = QtWidgets.QApplication(sys.argv)
memo = MyMemo()
sys.exit(app.exec_())

- ```def __init__(self, parent, bg_color):``` 타이틀바는 부모(메모) 윈도우에 종속되는 객체기 때문에 부모 윈도우로 접근이 가능해야 합니다. 그래서 생성시에 parent 변수를 선언하여 부모가 타이틀바를 생성시에 ```self.title_bar = MyBar(self)``` 이렇게 부모 정보를 넘겨줄 수 있도록 합니다. 

- 타이틀바에는 새로운 메모를 생성할 수 있는 버튼(+), 색상을 변경할 수 있는 버튼(c), 메모창을 닫을 수 있는 버튼(x) 을 QPushButton 을 생성하고 버튼 사이의 공간을 확보하기 위해 QLabel 을 사용하도록 하였습니다. 

- 각 위젯의 색상은 부모(메모)위젯의 색상과 동일해야 하기 때문에 ```self.btn_new.setStyleSheet("border:0px; background-color: {};".format(parent.color_bg))``` 처럼 ```parent.color_bg``` 로 부모의 색상값을 가져와 설정합니다.

- 이 위젯들은 QHBoxLayout 을 사용해서 타이틀바 윈도우 전체에 공백과 여백없이 레이아웃을 설정합니다.